### Matching

In [1]:
import networkx as nx
from scipy.spatial import distance
from tqdm import tqdm
import pandas as pd
import numpy as np

import concurrent.futures

pd.set_option('display.max_columns', None)

In [2]:
# Cols that we want to Match
cols_m = [
    'log Area Comercial',
    'log Area Residencial - V',
    'log Area Residencial - H',
    'log Area Terreno',
    'CA'
]

In [3]:
# Read the data
df_results = pd.read_parquet('df_results.parquet')
df_results.sample(10)

,sq,geometry,Nome,Linha,elegivel,T,Inauguração,ANO DO EXERCICIO,AREA OCUPADA,AREA DO TERRENO,AREA CONSTRUIDA,Area Comercial,Area Residencial - H,Area Residencial - V,Area Terreno,Residencial - H,Residencial - V,Comercial,Terreno,QUANTIDADE DE PAVIMENTOS,VALOR DO M2 DE CONSTRUCAO,VALOR DO M2 DO TERRENO,Valor,ANO DE INICIO DA VIDA DO CONTRIBUINTE,CA,TO,ano,delta,per Area Comercial,per Area Residencial - V,per Area Residencial - H,diff AREA CONSTRUIDA,diff Area Comercial,diff Area Residencial - H,diff Area Residencial - V,sum AREA CONSTRUIDA,sum Area Comercial,sum Area Residencial - H,sum Area Residencial - V,log AREA CONSTRUIDA,log Area Comercial,log Area Residencial - H,log Area Residencial - V,log Area Terreno
263861,101315,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x002\x00\x0...",São Paulo-Morumbi,4,1,0,27/10/2018,1996,2208,5270,2713,0,2532,0,459,0.851852,0.000000,0.000000,0.074074,1.233690,163.925926,177.333333,0.487325,1965.326944,0.514801,0.418975,2018,-22,0.000000,0.000000,0.933284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.905810,0.000000,7.836765,0.000000,6.129050
762598,167122,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00*\x00\x0...,Capão Redondo,5,1,0,20/10/2002,2003,1162,2866,1346,258,1087,0,0,0.846154,0.000000,0.153846,0.000000,1.073551,280.153846,38.000000,0.405019,1964.988856,0.469644,0.405443,2002,1,0.191679,0.000000,0.807578,0.0,0.0,0.0,0.0,230.0,122.0,108.0,0.0,7.204893,5.552960,6.991177,0.000000,0.000000
351642,068311,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x11\x00...,Tucuruvi,1,1,0,29/04/1998,2008,502,874,821,0,820,0,0,1.000000,0.000000,0.000000,0.000000,1.510353,492.000000,225.000000,0.403440,1960.609013,0.939359,0.574371,1998,10,0.000000,0.000000,0.998782,0.0,0.0,0.0,0.0,268.0,0.0,268.0,0.0,6.710523,0.000000,6.709304,0.000000,0.000000
87936,299015,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00k\x00\x0...,Fradique Coutinho,4,1,0,15/11/2014,1996,10527,32984,59618,2408,3969,48554,5986,0.509091,0.127273,0.200000,0.072727,12.394076,240.987413,359.236364,29.684127,1988.035543,1.807482,0.319155,2014,-18,0.040390,0.814418,0.066574,7995.0,-1292.0,305.0,8972.0,7995.0,-1292.0,305.0,8972.0,10.995713,7.786552,8.286269,10.790432,8.697179
465526,052256,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00d\x00\x0...,Vila Prudente,2,1,0,21/08/2010,2004,7301,9816,11845,5175,6237,432,0,0.646154,0.015385,0.338462,0.000000,2.006838,405.123077,242.000000,4.975994,1967.119966,1.206703,0.743786,2010,-6,0.436893,0.036471,0.526551,0.0,0.0,0.0,0.0,851.0,730.0,121.0,0.0,9.379661,8.551595,8.738255,6.068426,0.000000
400119,066158,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xbc\x00...,Parada Inglesa,1,1,0,29/04/1998,2012,9504,15695,13812,1507,11217,0,0,0.916667,0.000000,0.055556,0.000000,1.848031,598.370370,241.972222,8.761179,1972.288083,0.880025,0.605543,1998,14,0.109108,0.000000,0.812120,0.0,0.0,0.0,0.0,4166.0,208.0,2871.0,0.0,9.533293,7.317876,9.325186,0.000000,0.000000
742182,171206,b'\x01\x06\x00\x00\x00\x03\x00\x00\x00\x01\x03...,Campo Limpo,5,1,0,20/10/2002,2005,10402,40708,83165,3646,0,72397,3817,0.000000,0.481481,0.185185,0.148148,14.914640,520.763591,357.851852,53.188510,1988.472579,2.042965,0.255527,2002,3,0.043841,0.870522,0.000000,50.0,0.0,0.0,19.0,20320.0,604.0,0.0,18200.0,11.328582,8.201386,0.000000,11.189920,8.247220
753467,165047,b'\x01\x06\x00\x00\x00\x03\x00\x00\x00\x01\x03...,Campo Limpo,5,1,0,20/10/2002,2010,11121,28899,14228,4517,9130,0,2017,0.820755,0.000000,0.103774,0.056604,1.507028,525.306604,145.698113,8.494883,1970.674445,0.492335,0.384823,2002,8,0.317473,0.000000,0.641692,186.0,200.0,-14.0,0.0,1865.0,248.0,1467.0,0.0,9.562967,8.415603,9.119321,0.000000,7.609367
640392,147186,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00G\x00\x0...,Jardim Planalto,15,1,0,26/08/2019,2018,3544,6452,4482,0,4481,0,520,0.945946,0.000000,0.000000,0.054054,1.599509,733.486486,569.378378,3.524759,1991.773985,0.694668,0.549287,2019,-1,0.000000,0.000000,0.999777,0.0,0.0,0.0,0.0,3436.0,0.0,3436.0,0.0,8.407825,0.000000,8.407602,0.000000,6.253829
726659,168040,b'\x01\x06\x00\x00\x00\x03\x00\x0

In [9]:
def matching_algo(treated, control):
    # Create an empty graph
    graph = nx.Graph()

    # Add treated units to the graph as nodes
    for _, treated_row in treated.iterrows():
        graph.add_node(treated_row.sq, bipartite=0)

    # Add control units to the graph as nodes
    for _, control_row in control.iterrows():
        graph.add_node(control_row.sq, bipartite=1)

    # We iterate: for each treated and for each control unitm
    # We calculate the distance in respespect over the macthing columns
    # The distante is the weight edge
    for _, treated_row in treated.iterrows():
        for _, control_row in control.iterrows():
            dist = distance.euclidean(treated_row[cols_m], control_row[cols_m])
            graph.add_edge(treated_row.sq, control_row.sq, weight=dist)
    
    # So we calculate the distance for each pair
    # So now we apply the algorithm
    # It'll return the perfect match which minimize the distance
    matching = nx.bipartite.minimum_weight_full_matching(graph)
    return matching

def finding_matching(df, nome):
    # Select the ids columns + Matching columns
    cols = ['Nome', 'sq', 'T'] + cols_m
    
    # Filter only the data we need
    # Group we want and the prior
    df_f = df_results[
        (df.delta == -3) 
        & (df.Nome == nome)
    ][cols]
    
    # Separate treated and control units    
    treated = df_f[df_f['T'] == 1]
    control = df_f[df_f['T'] == 0]
    
    # Apply Matching algorithm
    matching = matching_algo(treated, control)
    
    # Return DataFrame
    df_matching = pd.concat([
        control[control.sq.isin([matching[e] for e in treated.sq])],
        treated
    ])
    return df_matching

In [10]:
# Selecting all the Metro Stations Groups to iterate over
aux_list_result = set(df_results.Nome.unique())

# This process takes timing
# To avoid that, we use a multiprocessing.
# That means, instead of iterating one at time, we have 8 workers
with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
    # Apply algorithm
    futures = [executor.submit(finding_matching, df_results, i) for i in aux_list_result]
    # Collecting the resulting and tracking the processing
    results = [future.result() for future in tqdm(concurrent.futures.as_completed(futures))]

39it [03:14,  4.99s/it]


In [11]:
# Concat the results into a unified dataframe
df_results_matching = pd.concat(results)

In [ ]:
# Lets see before the matching
pd.DataFrame(df_results[df_results.delta == -3].groupby(['T'])[cols_m].mean())

,log Area Comercial,log Area Residencial - V,log Area Residencial - H,log Area Terreno,CA
T,,,,,
0,4.905697,1.562889,7.262985,3.776681,0.846925
1,6.067352,2.794453,6.955426,2.878183,1.075458


In [14]:
# Apply an inner join to select only the matched units
df_results_matching_all = pd.merge(
    df_results,
    df_results_matching[['Nome', 'sq', 'T']],
    on = ['Nome', 'sq', 'T']
)
# Let's compare after matching
df_results_matching_all[df_results_matching_all.delta == -3].groupby(['T'])[cols_m].mean()

,log Area Comercial,log Area Residencial - V,log Area Residencial - H,log Area Terreno,CA
T,,,,,
0,6.124670,2.860344,7.051585,2.878016,1.037066
1,6.079256,2.911418,6.919388,2.887065,1.102270


In [ ]:
df_results_matching_all.to_parquet('did_matching.parquet')